### Modele współbieżności - żargon

- Równoległość - Możliwość wykonywania wielu obliczeń w tym samym czasie. Wymaga to wielordzeniowego procesora, wielu procesorów, procesora graficznego lub wielu komputerów w klastrze.
- Współbieżność - Możliwość obsługi wielu oczekujących zadań, dokonywanie postępów po jednym na raz lub równolegle (jeśli to możliwe), tak aby każdy z nich zakończył się ostatecznie sukcesem lub porażką. Jednordzeniowy procesor może obsługiwać współbieżność, jeśli wykonuje program dyspozytora z systemu operacyjnego, który przeplata wykonywanie oczekujących zadań. 
- Jednostka wykonawcza - Ogólne określenie obiektów, które wykonują kod współbieżnie. Python obsługuje natywnie trzy rodzaje jednostek wykonawczych: procesy, wątki i współprogramy.
- Proces - Wystąpienie programu komputerowego podczas jego działania, wykorzystujące pamięć i wycinek czasu procesora. Procesy komunikują się za pomocą potoków, gniazd lub plików mapowanych w pamięci. Wszystkie te elementy mogą przenosić tylko surowe bajty, dlatego też obiekty Pythona muszą myć serializowalne (konwertowalne) na surowe bajty aby mogły przechodzić z jednego procesu do drugiego. Jest to kosztowne i nie wszystkie obiekty można serializować.
- Wątek - Jednostka wykonawcza w ramach pojedynczego procesu. Gdy proces się zaczyna, wykorzystuje jeden wątek: wątek główny. Proces może tworzyć więcej wątków do współbieżnego działania, wywołując API systemu operacyjnego. To pozwala na łatwe udostępnianie danych pomiędzy wątkami ale może prowadzić do uszkodzenia danych gdy więcej niż jeden wątek aktualizuje współbieżnie ten sam obiekt. 
- Współprogram - Funkcja która może zawieszać swoje działanie i wznawiać je później. W Pythonie klasyczne współprogramy są budowane z funkcji generatorów a natywne współprogramy są definiowane przy pomocy `async def`. Współprogramy wspierają kooperacyjną wielozadaniowość: każdy współprogram musi jawnie oddać sterowanie przy użyciu słowa kluczowego `yield` lub `await` aby inny mógł działać współbieżnie (ale nie równolegle).
- Kolejka - Struktura danych, która pozwala umieszczać i pobierać elementy, zwykle w kolejności FIFO. Kolejki umożliwiają oddzielnym jednostkom wykonawczym wymianę danych aplikacji i komunikatów sterujących.
- Blokada - Obiekt, który może być wykorzystywany przez jednostki wykonawcze do synchronizowania swoich działań i unikania uszkadzania danych.
- Rywalizacja - Spór o ograniczony zasób. Rywalizacja występuje gdy wiele jednostek wykonawczych próbuje uzyskać dostęp do wspólnego zasobu. Istnieje też rywalizacja o procesor, gdy procesy lub wątki intensywne obliczeniowo muszą czekać aż dyspozytor systemu operacyjnego przydzieli im udział w czasie procesora.

### Procesy, wątki i globalna blokada GIL (Global Interpreter Lock)

1. Każde wystąpienie interpretera Pythona jest procesem. Możemy uruchamiać dodatkowe procesy Pythona korzystając z biblioteki `multiprocessing`.
2. Interpreter wykorzystuje jeden wątek do wykonywania programu i odśmiecania pamięci. Możemy uruchamiać dodatkowe wątki wykorzystując bibliotekę `threading`.
3. Tylko jeden wątek może utrzymywać blokadę GIL w danym momencie. Oznacza to, że tylko jeden wątek może wykonywać kod w danym momencie. 
4. Aby zapobiec utrzymywaniu blokady GIL przez jakiś wątek bez końca, interpreter wstrzymuje bieżący wątek domyślnie co 5 ms, zwalniając blokadę GIL.
5. Gdy piszemy kod w Pythonie, nie mamy kontroli nad blokadą GIL. Jednak funkcja wbudowana lub rozszerzenie napisane w C, może zwolnić blokadę.
6. Każda funkcja biblioteki standardowej, która wykonuje wywołanie syscall, zwalnia blokadę GIL. Obejmuje to wszystkie funkcje, które przeprowadzają operacje I/O, sieciowe i `time.sleep()`.
7. Rozszerzenia które integrują się na poziomie API języków Python/C mogą uruchamiać inne wątki niepythonowe, na które blokada GIL nie ma wpływu.
8. Wpływ blokady GIL na programowanie sieciowe jest niewielki, ponieważ funkcje I/O zwalaniają blokadę a zapisywanie i odczytywanie w sieci zawsze oznacza znaczne opóźnienia w stosunku do zapisywania i odczytywania w pamięci. W związku z tym każdy wątek poświęca dużo czasu na oczekiwanie.
9. Rywalizacja o blokadę GIL spowalnia intensywane obliczeniowo wątki Pythona. Sekwencyjny, jednowątkowy kod jest prostszy i szybszy w takich zadaniach.
10. Aby uruchomić kod intensywnie korzystający z procesora na wielu rdzeniach, musimy korzystać z wielu procesów Pythona.